In [ ]:
from nbtemplate import display_header
display_header('MirrorEffectiveArea.ipynb')

In [ ]:
from copy import deepcopy

import numpy as np
from astropy.table import QTable
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.visualization import ImageNormalize, MinMaxInterval, LogStretch, AsymmetricPercentileInterval, SqrtStretch
from scipy import stats
from marxs.source import PointSource, FixedPointing
from marxs.missions.AXIS import AXIS as axis

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

## Effective area

The current instrument model contains a number of simplifications. The most important one is probably for the mirror itself. The mirror is modeled as plane onf infinitesimal thickeness. Each photon that intersects that plane is focussed perfectly in the focal point. There is no modelleing of individual mirror shells and no modelling of the double bounce in the WS geomtry. Instead, we simply add a Gaussian blur in in-plane and out-or-plane scattering that gives us a distribution of photon directions (and thus PSF) with the width that we expect the true PSF to have. The radius of the mirror modules is accurate (so the extend of the areas that needs to be covered with gratings is), but individual plates, thickness, etc and the support spider are not modelled. Thus the geometric opening area of the mirror is larger than it actually will be. 

I'm assuming a the mirror shells to be Ir coated and use the CXRO Henke tables for bulk iridium to calculate the reflection efficiency. Since I don't know the preliminary specifications for the AXIS cameras (at they are certainly subject to change), I base the filters and CCD QE on numbers that I've used in similar form for a number of other missions which I ray-traced in the past like Lynx, Arcus, and RedSox:

- Optical blocking filter: 40 nm Al, could be directly deposited on CCDs
- UV blocking filter: 30 nm Al + 45 nm polyimide
- Filter support: Si mesh with 95 % open area (typical value for Luxel filters)
- contamination on CCDs and filters: about 98% transmisivvity for all energies
- CCD QE from Suzaku XIS1 BI CCD, assuming 42 micron depletion depth

I'm assuming the same filters and efficiency for the zero-order detector and the dispersed signal. The exact shape of the affective area curve will depend on the numbers I have chosen here, but the assumptions are likely not widely off.

## A scale factor to account for geometric opening area of the mirror modules

In [ ]:
conf = deepcopy(axis.conf)
conf['mirrorgeometricscaling'] = 1
instrum_nogratings = axis.PerfectAXIS(gratings=False, conf=conf)

In [ ]:
coord = SkyCoord(123 * u.deg, -23 * u.deg)
source = PointSource(coords=coord, energy=1 * u.keV)
pointing = FixedPointing(coords=coord)

In [ ]:
photons = source.generate_photons(1e5 * u.s)
photons = pointing(photons)

In [ ]:
photons = instrum_nogratings(photons)

In [ ]:
instrum_nogratings.elements[0].area

In [ ]:
aeff_sim_simple = (photons['probability'].sum() / len(photons) * instrum_nogratings.elements[0].area).to(u.cm**2)
aeff_sim_simple

Until the mirror geometry is modelled in more detail, I will simply determine a scale factor at 1 keV between my simulation and some assumed value for the effective area at 1 keV; this number can be thought of as the fraction of the geometric area of a module that is not actively concentrating light, e.g. because it hits the 0.5 mm thick "side" of a Si shell or because it hits some part of the mirror support structure.

**In that sense, all other simulations are scaled to a zero-order effective area of 7000 cm$^2$ at 1 keV.** If that area goes up or down, the effective area for the dispersed spectra needs to be scaled in the same way.

In [ ]:
scale_factor = 7000 * u.cm**2 / aeff_sim_simple
scale_factor

This number is set as "mirrorgeometricscaling" in the conf director for AXIS and will thus be applied to all future simulations.

## Zero-order effective area

The following runs include the scale factor and give the zero order effective area over a wide band-pass.

In [ ]:
instrum_nogratings = axis.PerfectAXIS(gratings=False)

energies = np.arange(.2, 10., .1) * u.keV
aeff = np.zeros(len(energies)) * u.cm**2

In [ ]:
for i, e in enumerate(energies):
    source.energy = e
    photons = source.generate_photons(1e5 * u.s)
    photons = pointing(photons)
    photons = instrum_nogratings(photons)
    aeff[i] = photons['probability'].sum() / len(photons) * instrum_nogratings.elements[0].area

In [ ]:
fig, ax = plt.subplots()
out = ax.loglog(energies, aeff)
ax.set_xlabel('energy (keV)')
out = ax.set_ylabel('effective area (cm$^2$)')